In [90]:
import pandas as pd
import os

In [91]:
df = pd.read_csv('./raw_data/' + os.listdir('raw_data')[0])

In [92]:
df.columns = [i.lower() for i in df.columns]

In [93]:
df.loc[0]

player                   1. Connor McDavid
team                                   EDM
age                                     22
date of birth                 Jan 13, 1997
country                             Canada
weight                     193 lbs - 88 kg
height                       6' 1 - 185 cm
pos                                      C
handed                                Left
drafted           1 - Round 1 - 2015 (EDM)
slide cand.                            NaN
waivers exempt                         NaN
arb. elig                              NaN
gp                                       0
g                                        0
a                                        0
p                                        0
p/gp                                  0.00
+/-                                      0
sh                                       0
sh%                                   0.00
toi                                    NaN
w                                        -
l          

In [94]:
df['player'] = [i.split(' ', maxsplit=1)[1] for i in df['player']]

In [95]:
df['date of birth'] = pd.to_datetime(df['date of birth'])

In [96]:
df['weight'] = [int(i.split(' ', maxsplit=1)[0]) for i in df['weight']]

In [97]:
df['height'] = [int(i.split('-')[1].split(' ')[1]) for i in df['height']]

In [115]:
df = df[df['pos'] != 'G']

df['pos'] = ['D' if i.__contains__('D') else 'F' for i in df['pos']]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [98]:
df['drafted'] = [i.split(' ')[0] for i in df['drafted']]

In [99]:
df['slide cand.'] = [1 if i == '✓' else 0 for i in df['slide cand.']]

In [100]:
df['waivers exempt'] = [1 if i == '✓' else 0 for i in df['waivers exempt']]

In [101]:
df['arb. elig'] = [1 if i == '✓' else 0 for i in df['arb. elig']]

In [102]:
drop_cols = [
    'gp', 'g', 'a', 'p', 'p/gp', '+/-', 'sh', 'sh%', 'toi',
    'w', 'l', 'so', 'gaa', 'sv%', 'signing date', 'arb. elig.1',
    'cap hit', 'salary', 'base salary', 'minors', 's.bonus', 'p.bonus',
    'arb. req', 'exp. year'
]

df = df.drop(drop_cols, axis=1)

In [103]:
type_map = {
    'Standard (1-way)': 'one_way',
    '35+ (1-way)': 'one_way',
    'Standard (2-way)': 'two_way',
    'Entry-Level': 'elc'
}

df['type'] = df['type'].map(type_map)

In [104]:
df['clause'] = df['clause'].fillna('none')

In [107]:
df['cap hit %'] = [float(i.replace('%','')) for i in df['cap hit %']]

In [111]:
df['aav'] = [float(i.replace('$','').replace(',','')) for i in df['aav']]

In [117]:
df.head()

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,waivers exempt,arb. elig,type,signing age,clause,length,signing,expiry,cap hit %,aav
0,Connor McDavid,EDM,22,1997-01-13,Canada,193,185,F,Left,1,...,0,0,one_way,20,none,8,RFA,UFA,16.7,12500000.0
1,Artemi Panarin,NYR,27,1991-10-30,Russia,168,180,F,Right,-,...,0,0,one_way,27,NMC,7,UFA,UFA,14.3,11642857.0
2,Auston Matthews,TOR,21,1997-09-17,United States,223,191,F,Left,1,...,0,0,one_way,21,none,5,RFA,UFA,14.6,11634000.0
3,Erik Karlsson,SJS,29,1990-05-31,Sweden,190,183,D,Right,15,...,0,0,one_way,29,NMC,8,UFA,UFA,14.5,11500000.0
4,Drew Doughty,LAK,29,1989-12-08,Canada,202,185,D,Right,2,...,0,0,one_way,29,NMC,8,UFA,UFA,13.8,11000000.0


In [120]:
df.loc[0]

player                 Connor McDavid
team                              EDM
age                                22
date of birth     1997-01-13 00:00:00
country                        Canada
weight                            193
height                            185
pos                                 F
handed                           Left
drafted                             1
slide cand.                         0
waivers exempt                      0
arb. elig                           0
type                          one_way
signing age                        20
clause                           none
length                              8
signing                           RFA
expiry                            UFA
cap hit %                        16.7
aav                          1.25e+07
Name: 0, dtype: object

# Combine with Career totals prior to signing